# Overview
Let's create an overview of the models we got so far. For each model, we'll apply 5-fold cross validation to get an average accuracy and standard deviation. We'll also test some different feature combinations.

In [1]:
from sklearn import metrics
from sklearn.model_selection import GroupKFold
from util import get_preprocessed_dataset, get_winners_only, fit_predict_print_wp, get_label_columns, get_embed_column_names, add_extra_features
import pandas as pd
import numpy as np

In [2]:
try:
    df = pd.read_pickle('df.pkl')
    extra_features = ["Length", "NumWordsDiff", "AvgWordLengthDiff", "MaxWordLengthDiff"] + get_embed_column_names()
except FileNotFoundError:
    df = get_preprocessed_dataset()
    df, extra_features = add_extra_features(df)
    df.to_pickle('df.pkl')

In [3]:
# Create a new datframe with columns model_name, manual_mean, manual_std, extra_mean, extra_std, embed_mean, embed_std, all_mean, all_std
feature_groups = [
    ('manual', get_label_columns()),
    ('extra', get_label_columns() + ['NumWordsDiff', 'AvgWordLengthDiff', 'MaxWordLengthDiff']),
    ('embed', get_embed_column_names()),
    ('all', get_label_columns() + ['NumWordsDiff', 'AvgWordLengthDiff', 'MaxWordLengthDiff'] + get_embed_column_names())
]

column_names = []
for group in feature_groups:
    column_names.append(group[0] + '_mean')
    column_names.append(group[0] + '_std')

evaluation_df = pd.DataFrame(columns=['model_name'] + column_names)

def evaluate_model(model_, model_name, fit_predict_wp_kwargs=None):
    if fit_predict_wp_kwargs is None:
        fit_predict_wp_kwargs = {}

    x = df.drop(['Winner'], axis=1)
    y = df # Taking winners only after train test split
    groups = df['Test']

    evaluation_results = {'model_name': model_name}
    for group_name, group_features in feature_groups:
        scores = []
        for train_index, test_index in GroupKFold(n_splits=20).split(x, y, groups):
            x_train, x_test = x.iloc[train_index], x.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            train_groups = groups.iloc[train_index]


            y_goal = get_winners_only(y_test)

            accuracy = fit_predict_print_wp(model_, x_train, y_train, x_test, y_goal, silent=True, features=group_features, groups=train_groups, **fit_predict_wp_kwargs)

            scores.append(accuracy)

        mean = np.mean(scores)
        std = np.std(scores)

        print(f"{model_name}[{group_name}] - \tMean: {mean:.3f}, Std: {std:.3f}, Min {np.min(scores):.3f}, Max {np.max(scores):.3f}")
        evaluation_results[f"{group_name}_mean"] = mean
        evaluation_results[f"{group_name}_std"] = std

    # Add the results as a new row in the evaluation dataframe
    evaluation_df.loc[len(evaluation_df)] = evaluation_results

## Random
The first classifier we tried, used as a baseline.

In [11]:
from util import get_random_predictor_model
model = get_random_predictor_model()
evaluate_model(model, 'Random')

Random[manual] - 	Mean: 0.458, Std: 0.075, Min 0.333, Max 0.609
Random[extra] - 	Mean: 0.446, Std: 0.065, Min 0.289, Max 0.556
Random[embed] - 	Mean: 0.445, Std: 0.045, Min 0.391, Max 0.543
Random[all] - 	Mean: 0.406, Std: 0.073, Min 0.267, Max 0.522


## Naive Bayes

In [4]:
from util import get_naive_bayes_model_wp
model = get_naive_bayes_model_wp()
evaluate_model(model, 'Naive Bayes')

Naive Bayes[manual] - 	Mean: 0.625, Std: 0.073, Min 0.457, Max 0.739
Naive Bayes[extra] - 	Mean: 0.614, Std: 0.065, Min 0.478, Max 0.717
Naive Bayes[embed] - 	Mean: 0.464, Std: 0.060, Min 0.348, Max 0.565
Naive Bayes[all] - 	Mean: 0.523, Std: 0.065, Min 0.391, Max 0.644


## Random Forest

In [6]:
from util import get_random_forest_model_wp
model = get_random_forest_model_wp()
evaluate_model(model, 'Random Forest') # Note: In lines below, there is still "Naive Bayes", but that's actually a random forest

Naive Bayes[manual] - 	Mean: 0.621, Std: 0.081, Min 0.413, Max 0.761
Naive Bayes[extra] - 	Mean: 0.615, Std: 0.072, Min 0.489, Max 0.739
Naive Bayes[embed] - 	Mean: 0.478, Std: 0.074, Min 0.261, Max 0.600
Naive Bayes[all] - 	Mean: 0.594, Std: 0.059, Min 0.435, Max 0.696


## Multi Layer Perceptron

In [4]:
from util import get_mlp_model_wp
model = get_mlp_model_wp()
evaluate_model(model, 'MLP')

MLP[manual] - 	Mean: 0.546, Std: 0.074, Min 0.413, Max 0.711
MLP[extra] - 	Mean: 0.562, Std: 0.088, Min 0.435, Max 0.756
MLP[embed] - 	Mean: 0.489, Std: 0.076, Min 0.304, Max 0.609
MLP[all] - 	Mean: 0.544, Std: 0.062, Min 0.370, Max 0.622


## Gradient Boosting

In [5]:
from util import get_xgboost_model_wp
model = get_xgboost_model_wp()
evaluate_model(model, 'XGBoost')

XGBoost[manual] - 	Mean: 0.575, Std: 0.081, Min 0.457, Max 0.761
XGBoost[extra] - 	Mean: 0.581, Std: 0.081, Min 0.457, Max 0.826
XGBoost[embed] - 	Mean: 0.436, Std: 0.074, Min 0.326, Max 0.600
XGBoost[all] - 	Mean: 0.586, Std: 0.051, Min 0.489, Max 0.667


## Proximity Forest

In [4]:
from util import get_proximity_forest_model_wp
model = get_proximity_forest_model_wp()
evaluate_model(model, 'Proximity Forest')

TypeError: fit() missing 1 required positional argument: 'groups'